<a href="https://colab.research.google.com/github/choarauc/form/blob/main/DL_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 définir notre modèle pour une grille de taille 8x8.

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, LeakyReLU, Dropout, Reshape, BatchNormalization
from tensorflow.keras import Sequential

nb_class = 0

# Backbone
efficientNet = EfficientNetB0(include_top=False, input_shape=(256,256,3))

# Freeze the blackbone
for layer in efficientNet.layers:
    layer.trainable = False

# Définition de la partie encoder
model = Sequential()
# Feature extration part
model.add(efficientNet) 
model.add(Reshape([-1, 1280]))
# Regression Part
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5 + nb_class))
model.summary()

Prédiction

onction convertissant la sortie du modèle en coordonnées (x,y,w,h)

In [ ]:
def transform_netout(y_pred_raw):
    y_pred_xy = (tf.nn.tanh(y_pred_raw[..., 1:3]))
    y_pred_wh = tf.sigmoid(y_pred_raw[..., 3:5])
    y_pred_conf = tf.sigmoid(y_pred_raw[..., :1])
    return tf.concat([y_pred_conf, y_pred_xy, y_pred_wh], -1)

DATASET

In [ ]:
import pandas as pd
df = pd.read_csv('face_images.csv')

df['xmoy'] = (df.xmax + df.xmin)/2
df['ymoy'] = (df.ymax + df.ymin)/2
df['w'] = (df.xmax - df.xmin)
df['h'] = (df.ymax - df.ymin)


df.head()


fonctions load_image et show_bounding_box.

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
folder_images = 'images/'

@tf.function
def load_image(filepath, resize=None):
    im = tf.io.read_file(folder_images + filepath)
    im = tf.image.decode_png(im, channels=3)
    if resize:
        return tf.image.resize(im, resize)
    else :
        return im


def show_bounding_box(im, bbox, normalised=True, color='r'):
    # Signification de bbox
    x, y, w, h = bbox
    # Convertir les cordonées (x,y,w,h) en (x1,x2,y1,y2)
    x1=x-w/2
    x2=x+w/2
    y1=y-h/2
    y2=y+h/2
    
    # redimensionner en cas de normalisation
    if normalised:
        x1=x1*im.shape[1]
        x2=x2*im.shape[1]
        y1=y1*im.shape[0]
        y2=y2*im.shape[0]
        
    # Afficher l'image
    plt.imshow(im)
    
    # Afficher la bounding box
    plt.plot([x1,x2,x2,x1,x1],[y1,y1,y2,y2,y1],"r")
        
im = load_image(df.filename[0])

bbox = df[['xmoy', 'ymoy', 'w', 'h']].values[0]
# Afficher l'image ainsi que la bounding box
show_bounding_box(im, bbox)

In [ ]:
from sklearn.model_selection import train_test_split

X_train_path, X_test_path, y_train, y_test = train_test_split(df.filename.values, df[['xmoy', 'ymoy', 'w', 'h']].values, train_size=0.9, random_state=1234)


Générateur et augmentation de données

In [ ]:
import tensorflow as tf
import numpy as np


def load_image(filepath, y, resize=(256,256)):
    # Load image
    im = tf.io.read_file(folder_images+filepath)
    im = tf.image.decode_png(im, channels=3)
    # Resize image
    im = tf.image.resize(im, resize)
    # Find the max translation to keep the object in the image
    tx_max = resize[1]*tf.nn.relu(y[0]-y[2]/2)
    tx_min = -resize[1]*tf.nn.relu(1-y[0]-y[2]/2)
    ty_max = resize[0]*tf.nn.relu(y[1]-y[3]/2)
    ty_min = -resize[0]*tf.nn.relu(1-y[1]-y[3]/2)
    tx = np.random.uniform(tx_min, tx_max)
    ty = np.random.uniform(ty_min, ty_max)
    # Apply the transformation in the image
    im = tf.keras.preprocessing.image.apply_affine_transform(
    im.numpy(), theta=0, tx=ty, ty=tx, shear=0, zx=1, zy=1, row_axis=0, col_axis=1,
    channel_axis=2, fill_mode='nearest', cval=0.0, order=1
)
    # Correct the target variable
    y_new = y.numpy().copy()
    y_new[0] += -tx/resize[1]
    y_new[1] += -ty/resize[0]
    
    return im, y_new



In [ ]:
# resultat de l'augmentation 


filepath = X_train_path[1]
bbox = y_train[1]


plt.figure(figsize=(15,7))
for j in range(8):
    plt.subplot(2,4,j+1)
    img, y = tf.py_function(load_image, [filepath, bbox], [tf.float32, tf.float32])
    show_bounding_box(img.numpy().astype(int), y)
    plt.xticks([])
    plt.yticks([])
    # plt.imshow(X_t[i].numpy().astype(int))

Variable cible


fonction convert_target.

In [ ]:
import tensorflow as tf
nb_class = 0
output_shape = (8, 8)

def convert_target(bboxes, output_shape=output_shape, nb_classes=0):
    y_target = np.zeros([output_shape[0], output_shape[1], 1+4])# +nb_classes
    lx=1/output_shape[1]
    ly=1/output_shape[0]
    for x, y, w, h in [bboxes]:
        idx_x = int(x//lx)
        idx_y = int(y//ly)
        # Presence of object
        y_target[idx_y, idx_x, 0] = 1
        # Class of object
        # y_target[idx_x, idx_y, 5+int(c)] = 1
        # Coordinate x
        y_target[idx_y, idx_x, 1] = 2*(x/lx - (idx_x+0.5))
        # Coordinate y
        y_target[idx_y, idx_x, 2] = 2*(y/ly - (idx_y+0.5))
        # Coordinate w
        y_target[idx_y, idx_x, 3] = w
        # Coordinate h
        y_target[idx_y, idx_x, 4] = h
        
    return y_target.reshape([-1, 1+4])# +nb_classes

convert_target(bbox, output_shape, nb_classes=nb_class)

 matrice de probabilité de présence.

In [ ]:
img, y = tf.py_function(load_image, [filepath, bbox], [tf.float32, tf.float32])
plt.figure(figsize=(12,5))
plt.subplot(121)
show_bounding_box(img.numpy().astype(int), y)
plt.subplot(122)
plt.imshow(convert_target(y, output_shape)[:,0].reshape(output_shape))

In [ ]:
# definition du dataset

batch_size = 64

def load_image(filepath, y, resize=(256,256)):
    im = tf.io.read_file(folder_images + filepath)
    im = tf.image.decode_png(im, channels=3)
#     im_shape = tf.shape(im)
    im = tf.image.resize(im, resize)
    tx_max = 256*tf.nn.relu(y[0]-y[2]/2)
    tx_min = -256*tf.nn.relu(1-y[0]-y[2]/2)
    ty_max = 256*tf.nn.relu(y[1]-y[3]/2)
    ty_min = -256*tf.nn.relu(1-y[1]-y[3]/2)
    tx = np.random.uniform(tx_min, tx_max)
    ty = np.random.uniform(ty_min, ty_max)
    im = tf.keras.preprocessing.image.apply_affine_transform(
    im.numpy(), theta=0, tx=ty, ty=tx, shear=0, zx=1, zy=1, row_axis=0, col_axis=1,
    channel_axis=2, fill_mode='nearest', cval=0.0, order=1
)
    y_new = y.numpy().copy()
    y_new[0] += -tx/256
    y_new[1] += -ty/256
    
    return im, convert_target(y_new, output_shape)


dataset_train = tf.data.Dataset.from_tensor_slices((X_train_path, y_train))

dataset_train = dataset_train.shuffle(100000).map(lambda x, y : tf.py_function(load_image, [x, y], [tf.float32, tf.float32]), num_parallel_calls=-1).batch(int(batch_size))

In [ ]:
# verif de la cohérence


X_t, y_t = next(iter(dataset_train))
plt.figure(figsize=(15,7))
for j, i in enumerate(np.random.randint(0, batch_size, [4])):
    plt.subplot(2,4,2*j+1)
    plt.imshow(X_t[i].numpy().astype(int))
    plt.subplot(2,4,2*j+2)
    plt.imshow(y_t[i,:,0].numpy().reshape(output_shape))


In [ ]:
y_pred_raw = model.predict(X_t)

y_pred = transform_netout(y_pred_raw)



Fonction de perte de l'algorithme

implémentation de cette fonction de perte.

In [ ]:
nb_class = 20

def class_loss(y_true, y_pred):
    # Vecteur de présence d'un objet
    y_true_conf = y_true[...,0]
    # Probabilité conditionelle des vrais objets
    y_true_class = tf.one_hot(y_true[...,5], nb_class)
    # Probabilité conditionelle des prédictions
    y_pred_class = y_pred[...,5:]
    # Calcul de la fonction de perte
    class_loss = tf.reduce_sum(y_true_conf*tf.reduce_sum(tf.squarre(y_true_class - y_pred_class), axis=-1), axis=-1)
    return class_loss

L'erreur de localisation

In [ ]:
def coord_loss(y_true, y_pred):
    # Probabilty of object presence
    y_true_conf = y_true[...,0]
    
    # x and y loss for real object
    y_true_xy = y_true[...,1:3]
    y_pred_xy = y_pred[...,1:3]
    xy_loss = tf.reduce_sum(tf.reduce_sum(tf.square(y_true_xy - y_pred_xy),axis=-1)*y_true_conf, axis=-1)
    
    # w and h loss for real object
    y_true_wh = y_true[...,3:5]
    y_pred_wh = y_pred[...,3:5]
    wh_loss = tf.reduce_sum(tf.reduce_sum(tf.square(tf.sqrt(y_true_wh) - tf.sqrt(y_pred_wh)), axis=-1)*y_true_conf, axis=-1)
    return xy_loss + wh_loss

# Coordinate loss for the prediction X_t
coord_loss(y_t, y_pred)


In [ ]:
lambda_noobj = 0.5

def object_loss(y_true, y_pred):
    # x and y loss for real object
    y_true_p = y_true[...,0]
    y_pred_p = y_pred[...,0]
    return tf.reduce_sum((lambda_noobj + (1-lambda_noobj)*y_true_p)*tf.square(y_true_p - y_pred_p), axis=-1)

# Object loss for the prediction X_t
object_loss(y_t, y_pred)


Erreur globale

In [ ]:
lambda_coord = 5
lambda_object = 1

def global_loss(y_true, y_pred):
    # Convert input
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_pred = transform_netout(y_pred)
    loss_coordinate = coord_loss(y_true, y_pred)
    loss_object = object_loss(y_true, y_pred)
    return lambda_object*loss_object + lambda_coord*loss_coordinate

# Global loss for the prediction X_t
global_loss(y_t, y_pred)

Entraînement du modèle

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import callbacks

model.compile(optimizer=Adam(1e-3), loss=global_loss)




# Save automatically the weights
checkpoint = callbacks.ModelCheckpoint(filepath = 'model', 
                                       monitor = 'loss',
                                       save_best_only = True,
                                       save_weights_only = True,
                                       mode = 'min',
                                       save_freq = 'epoch')

# Callback to reduce automatically the learning rate.
lr_plateau = callbacks.ReduceLROnPlateau(monitor = 'loss',
                                         patience=5,
                                         factor=0.1,
                                         verbose=2,
                                         mode='min')

model.fit(dataset_train, epochs=10, workers=-1,  callbacks = [lr_plateau, checkpoint])

Prédiction des coordonnées absolues

In [ ]:
def generate_yolo_grid(g):
    c_x = tf.cast(tf.reshape(tf.tile(tf.range(g), [g]), (1, g, g)), 'float32')
    c_y = tf.transpose(c_x, (0,2,1))
    return tf.stack([tf.reshape(c_x, (-1, g*g)), tf.reshape(c_y, (-1, g*g))] , -1)

c_grid = generate_yolo_grid(output_shape[0])


def proccess_xy(y_true_raw):
    y_true_xy = ((y_true_raw[..., 1:3]+1)/2 + c_grid)/output_shape[0]
    y_true_wh = y_true_raw[..., 3:5]
    y_true_conf = y_true_raw[..., :1]
    return tf.concat([y_true_conf, y_true_xy, y_true_wh], -1)

Prédiction des boundings boxes les plus probables

In [ ]:
def pred_bboxes(y, threshold=0.3):
    y_xy = tf.cast(y, tf.float32)
    y_xy = tf.expand_dims(y_xy, axis=0)
    y_xy = proccess_xy(y_xy)[0]
    #return y_xy
    bboxes =  sorted(y_xy.numpy(), key=lambda x: x[0], reverse=True)
    bboxes = np.array(bboxes)
    result = bboxes[bboxes[:,0]>threshold]
    if len(result)== 0:
        return bboxes[[0]]
    return result       


In [ ]:
# test sur image internet 

import time
import cv2, urllib, json

def show_img(img, model, threshold=0.2):
    pred = model(np.array([img], dtype=np.float32))[0]
    pred = transform_netout(pred)


    bboxes_pred = pred_bboxes(pred, threshold)
    plt.imshow(X_t[0]/255)
    for bbox in bboxes_pred:
        bbox = bbox[1:]
        show_bounding_box(img/255, bbox)
    
def url_to_image(url):
    resp = urllib.request.urlopen(url) 
    img = np.asarray(bytearray(resp.read()), dtype="uint8")
    img = cv2.imdecode(img, -1)
    img = cv2.resize(img, (256,256))
    img = img[..., [2,1,0]]
    return tf.keras.applications.efficientnet.preprocess_input(img)

# Exemple
img = url_to_image("https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/iron-man-black-widow-marvel-1569327823.jpg")
show_img(img , model, 0.3)

In [ ]:
# si mlauvais result 


model = tf.keras.models.load_model('model_yolo', compile=False)

In [ ]:
# sur webcam

%matplotlib inline
from IPython.display import HTML
from PIL import Image
import base64, io
import numpy as np

main_text = """
<style type="text/css">
    canvas {
        display: none;
    }
    </style>

<video id="video" width="320" height="240" autoplay></video>
<button id="snap">Snap Photo</button>
<canvas id="canvas" width="320" height="240"></canvas>

<script>
// Grab elements, create settings, etc.
var video = document.getElementById('video');

// Get access to the camera!
if(navigator.mediaDevices && navigator.mediaDevices.getUserMedia) {
    // Not adding `{ audio: true }` since we only want video now
    navigator.mediaDevices.getUserMedia({ video: true }).then(function(stream) {
        //video.src = window.URL.createObjectURL(stream);
        //video.play();
        video.srcObject=stream;
        video.play();
    });
}

// Elements for taking the snapshot
var canvas = document.getElementById('canvas');
var context = canvas.getContext('2d');
var video = document.getElementById('video');
var image = canvas.toDataURL("image/png");
IPython.notebook.kernel.execute("image = '" + image + "'")

// Trigger photo take
document.getElementById("snap").addEventListener("click", function() {
	context.drawImage(video, 0, 0, 320, 240);
    var myCanvas = document.getElementById('canvas');
    var image = myCanvas.toDataURL("image/png");
    IPython.notebook.kernel.execute("print('testing')")
    IPython.notebook.kernel.execute("imageWebCam = '" + image + "'")
});
</script>

"""

def show_bboxes_webcam(model):
    img = np.array(Image.open(io.BytesIO(base64.b64decode(imageWebCam.split(',')[1]))))[:,:,0:3]
    img = tf.image.resize(img, (256,256))
    show_img(img, model)
    
HTML(main_text)

In [ ]:
# Une fois le bouton "Snap Photo" pressé, exécuter la cellule suivante.

In [ ]:
# UNIQUEMENT EN LOCAL

import cv2
import numpy as np
# Load the cascade

# To capture video from webcam. 
cap = cv2.VideoCapture(0)
# To use a video file as input 
# cap = cv2.VideoCapture('filename.mp4')
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,500)
fontScale              = 1
fontColor              = (0,255,0)
lineType               = 2


while True:
    # Read the frame
    try :
        _, img = cap.read()
        # img = img[60:, 100:][:-60, :-100]
        
        img_resized = cv2.resize(img, (256, 256))
        
        prediction = model(np.array([img_resized[:,:,[2,1,0]]], dtype=np.float32))[0]
        
        prediction = transform_netout(prediction)
        
        bboxes = pred_bboxes(prediction, 0.05)
        
        # Convert to grayscale
    #     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Detect the faces
    #     faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        # Draw the rectangle around each face
        for c, x_m, y_m, w, h in bboxes:
            x_m = x_m * img.shape[1]
            y_m = y_m * img.shape[0]
            w = w * img.shape[1]
            h = h * img.shape[0]
            cv2.rectangle(img, (int(x_m-w/2), int(y_m-h/2)), (int(x_m+w/2), int(y_m+h/2)), fontColor, 2)
            cv2.putText(img, str(c)[:5], 
                        (int(x_m-w/2), int(y_m-h/2)+30), 
                        font, 
                        fontScale,
                        fontColor,
                        lineType)
        # Display
        
        cv2.imshow('image', img)
        # Stop if escape key is pressed
        k = cv2.waitKey(30) & 0xff
        if k==27:
            break
    except :
        break
# Release the VideoCapture object
cap.release()